<a href="https://colab.research.google.com/github/reallylongaddress/abnb_preprocessor/blob/master/notebooks/re_price_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import requests
from bs4 import BeautifulSoup, Tag
import pandas as pd
from google.colab import drive


In [92]:

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

# Scrape URL: 
provinces_url = 'https://www.immobiliare.it/en/mercato-immobiliare/'

# References
#https://hackersandslackers.com/scraping-urls-with-beautifulsoup

In [93]:
req = requests.get(provinces_url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
#print(soup.prettify())

In [94]:
drive.mount('/drive')


Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


# Load Regions

In [95]:
#look for 'table' class: nd-table nd-table--borderBottom
region_table = soup.find_all("table", class_="nd-table nd-table--borderBottom")

#look for the 'tr' class: nd-table__row
regions = []
for regions_tag in soup.find_all("tr", class_="nd-table__row"):
    
    region = ''
    url = ''
    sale = 0
    rent = 0

    for regions_tag_child in regions_tag.children:
        if type(regions_tag_child) == Tag:
            if regions_tag_child.get("data-text") == "(€/m²)":
                link = regions_tag_child.find('a')
                if (link):
                    url = link.get('href')
                    region_name = link.text
            elif regions_tag_child.get("data-text") == "Sale":
                sale = regions_tag_child.text.replace('.', '')
            elif regions_tag_child.get("data-text") == "Rent":
                rent = regions_tag_child.text.replace(',', '.')
                
    regions.append([region_name, sale, rent, url])

regions_df = pd.DataFrame(data = regions, columns=['region', 'sale_mean', 'rent_mean', 'url'])
regions_df['sale_mean'] = regions_df['sale_mean'].astype(int)
regions_df['rent_mean'] = regions_df['rent_mean'].astype(float)
regions_df

,region,sale_mean,rent_mean,url
0,Abruzzo,1296,8.00,https://www.immobiliare.it/en/mercato-immobili...
1,Basilicata,1369,7.26,https://www.immobiliare.it/en/mercato-immobili...
2,Calabria,922,7.30,https://www.immobiliare.it/en/mercato-immobili...
3,Campania,1882,9.34,https://www.immobiliare.it/en/mercato-immobili...
4,Emilia Romagna,1839,12.29,https://www.immobiliare.it/en/mercato-immobili...
5,Friuli Venezia Giulia,1478,8.96,https://www.immobiliare.it/en/mercato-immobili...
6,Lazio,2453,13.55,https://www.immobiliare.it/en/mercato-immobili...
7,Liguria,2526,10.41,https://www.immobiliare.it/en/mercato-immobili...
8,Lombardia,2157,15.58,https://www.immobiliare.it/en/mercato-immobili...
9,Marche,1520,9.18,https://www.immobiliare.it/en/mercato-immobili...


In [96]:
# for index, row in regions_df.iterrows():
#     print(f'{index}::{round((((row["rent"]*12)/row["sale"])*100),1)}::{row["region"]}')

In [97]:
# for index, row in regions_df.iterrows():
#     print(f'{index} {row["sale"]}/{round(row["rent"]*12, 0)}     {round(((row["sale"]/(row["rent"]*12))*100),1)}::{row["region"]}')
    
#     # Median Home Price / Median Annual Rent = Price to Rent Ratio.

# Load Region Detail and Provinces

In [98]:
regions_data = []
provinces_data = []
# print(f'shape: {regions_df.shape}')
for index, region in regions_df.iterrows():
    print(f'{index}: {region["url"]}')
    
    region_req = requests.get(region["url"], headers)
    region_soup = BeautifulSoup(region_req.content, 'html.parser')    
    
    #load the region price data
    
    region_prices = region_soup.find_all("p", class_="nd-cgHighlighted__subtext")
    
    #SALE 
    #break the string into words
    region_sale_price_parts = region_prices[0].contents[0].split()
    
    #keep only the parts we want
    region_sale_price_parts = [region_sale_price_parts[i] for i in [1,4]]

    #remove '.' thousands separator
    region_sale_price_parts = [price.replace('.', '') for price in region_sale_price_parts]
    # print(f'Sale: {region_sale_price_parts}')
    
    #RENT 
    #break the string into words
    region_rent_price_parts = region_prices[1].contents[0].split()
    
    #keep only the parts we want
    region_rent_price_parts = [region_rent_price_parts[i] for i in [1,4]]

    #remove '.' thousands separator
    region_rent_price_parts = [price.replace(',', '.') for price in region_rent_price_parts]
    
    regions_data.append([index, region_sale_price_parts[0], region_sale_price_parts[1], 
                         region_rent_price_parts[0], region_rent_price_parts[1]])

    if region['region'] != "Valle d'Aosta":
      print(f"regions_data['region'] != Valle dAosta")
    else:
      print(f"regions_data['region'] ===== Valle dAosta")      

    #Now load the province data
    province_soup = region_soup.find_all("tr", class_="nd-table__row")

    for province_html in province_soup:
        province_name = ''
        url = ''
        sale = 0
        rent = 0

        if region['region'] != "Valle d'Aosta":
          print(f"regions_data['region'] != Valle dAosta")
          for child in province_html.children:
              if type(child) == Tag:
                  if child.get("data-text") == "(€/m²)":
                      link = child.find('a')
                      if (link):
                          url = link.get('href')
                          province_name = link.text
                          print(f'province_name: {province_name}')
                  elif child.get("data-text") == "Sale":
                      sale = child.text.replace('.', '')
                  elif child.get("data-text") == "Rent":
                      rent = child.text.replace(',', '.')
          provinces_data.append([region['region'], province_name, sale, rent, url])            
          print(f'provinces_data: {len(provinces_data)}')

        else:
          print(f"regions_data['region'] ===== Valle dAosta") 
#merge region dataframes, drop index, reorder columns
regions_data_df = pd.DataFrame(data = regions_data, columns=['index', 
                                                             'sale_low', 'sale_high', 
                                                             'rent_low', 'rent_high'])
regions_df.reset_index(inplace=True)
print(f'A:  {regions_df.columns}')
regions_df = regions_df.merge(regions_data_df, left_on='index', right_on='index')
print(f'B:  {regions_df.columns}')
regions_df.drop(columns=['index'], inplace=True)
regions_df = regions_df[['region', 'sale_low', 'sale_mean', 'sale_high', 'rent_low', 'rent_mean', 'rent_high', 'url']]

provinces_data_df = pd.DataFrame(data=provinces_data, columns=['region', 'province', 'sale_mean', 
                                                               'rent_mean', 'url'])
                                 
# display(regions_df)
display(provinces_data_df)                                 

0: https://www.immobiliare.it/en/mercato-immobiliare/abruzzo/
regions_data['region'] != Valle dAosta
regions_data['region'] != Valle dAosta
province_name: Chieti
provinces_data: 1
regions_data['region'] != Valle dAosta
province_name: L'Aquila
provinces_data: 2
regions_data['region'] != Valle dAosta
province_name: Pescara
provinces_data: 3
regions_data['region'] != Valle dAosta
province_name: Teramo
provinces_data: 4
1: https://www.immobiliare.it/en/mercato-immobiliare/basilicata/
regions_data['region'] != Valle dAosta
regions_data['region'] != Valle dAosta
province_name: Matera
provinces_data: 5
regions_data['region'] != Valle dAosta
province_name: Potenza
provinces_data: 6
2: https://www.immobiliare.it/en/mercato-immobiliare/calabria/
regions_data['region'] != Valle dAosta
regions_data['region'] != Valle dAosta
province_name: Catanzaro
provinces_data: 7
regions_data['region'] != Valle dAosta
province_name: Cosenza
provinces_data: 8
regions_data['region'] != Valle dAosta
province_name:

,region,province,sale_mean,rent_mean,url
0,Abruzzo,Chieti,1133,6.62,https://www.immobiliare.it/en/mercato-immobili...
1,Abruzzo,L'Aquila,1220,7.62,https://www.immobiliare.it/en/mercato-immobili...
2,Abruzzo,Pescara,1433,8.48,https://www.immobiliare.it/en/mercato-immobili...
3,Abruzzo,Teramo,1384,10.55,https://www.immobiliare.it/en/mercato-immobili...
4,Basilicata,Matera,1536,8.09,https://www.immobiliare.it/en/mercato-immobili...
...,...,...,...,...,...
102,Veneto,Rovigo,974,7.40,https://www.immobiliare.it/en/mercato-immobili...
103,Veneto,Treviso,1561,9.29,https://www.immobiliare.it/en/mercato-immobili...
104,Veneto,Venezia,2337,13.09,https://www.immobiliare.it/en/mercato-immobili...
105,Veneto,Verona,1936,10.07,https://www.immobiliare.it/en/mercato-immobili...


In [99]:
provinces_data_df['region'].value_counts()

Lombardia                12
Toscana                  10
Emilia Romagna           10
Sicilia                   9
Piemonte                  8
Veneto                    7
Puglia                    6
Sardegna                  5
Marche                    5
Lazio                     5
Campania                  5
Calabria                  5
Liguria                   4
Friuli Venezia Giulia     4
Abruzzo                   4
Basilicata                2
Molise                    2
Trentino Alto Adige       2
Umbria                    2
Name: region, dtype: int64

In [100]:
provinces_data_df

,region,province,sale_mean,rent_mean,url
0,Abruzzo,Chieti,1133,6.62,https://www.immobiliare.it/en/mercato-immobili...
1,Abruzzo,L'Aquila,1220,7.62,https://www.immobiliare.it/en/mercato-immobili...
2,Abruzzo,Pescara,1433,8.48,https://www.immobiliare.it/en/mercato-immobili...
3,Abruzzo,Teramo,1384,10.55,https://www.immobiliare.it/en/mercato-immobili...
4,Basilicata,Matera,1536,8.09,https://www.immobiliare.it/en/mercato-immobili...
...,...,...,...,...,...
102,Veneto,Rovigo,974,7.40,https://www.immobiliare.it/en/mercato-immobili...
103,Veneto,Treviso,1561,9.29,https://www.immobiliare.it/en/mercato-immobili...
104,Veneto,Venezia,2337,13.09,https://www.immobiliare.it/en/mercato-immobili...
105,Veneto,Verona,1936,10.07,https://www.immobiliare.it/en/mercato-immobili...


In [101]:
regions_df.to_csv('/drive/My Drive/Colab_Data/AirBnb/Italy/Sicily/derived/re_regions.csv')

In [102]:
provinces_data_df.to_csv('/drive/My Drive/Colab_Data/AirBnb/Italy/Sicily/derived/re_provinces_data_df.csv')

In [103]:
regions_df.to_csv('/drive/My Drive/Colab_Data/AirBnb/Italy/Sicily/derived/re_regions.csv')

# Load Province Detail and Municipalities

In [104]:
# provinces_data_df[provinces_data_df['region'] == 'Veneto']


In [105]:
# abruzzo = provinces_data_df[provinces_data_df['region'] == 'Abruzzo']
# abruzzo
# provinces_data_df['region'].value_counts()

In [106]:
# province_data = []
# municipality_data = []

# # print(f'shape: {regions_df.shape}')
# for index, province in provinces_data_df[provinces_data_df['region'] == "Valle d'Aosta"].iterrows():
# # for index, province in provinces_data_df.iterrows():
#     print(f'{index}: {province["url"]}')
    
#     province_req = requests.get(province["url"], headers)
#     province_soup = BeautifulSoup(province_req.content, 'html.parser')    
    
#     #load the province price data
    
#     province_prices = province_soup.find_all("p", class_="nd-cgHighlighted__subtext")
    
#     #SALE 
#     #break the string into words
#     province_sale_price_parts = province_prices[0].contents[0].split()
    
#     #keep only the parts we want
#     province_sale_price_parts
#     # print(f'province_sale_price_parts: {province_sale_price_parts}')
#     if len(province_sale_price_parts) == 6:
#       province_sale_price_parts = [province_sale_price_parts[i] for i in [1,4]]

#       #remove '.' thousands separator
#       province_sale_price_parts = [price.replace('.', '') for price in province_sale_price_parts]
#       # print(f'Sale: {province_sale_price_parts}')
      
#       #RENT 
#       #break the string into words
#       province_rent_price_parts = province_prices[1].contents[0].split()
      
#       #keep only the parts we want
#       province_rent_price_parts = [province_rent_price_parts[i] for i in [1,4]]
#       # print(f'Rent: {province_rent_price_parts}')

#       #remove '.' thousands separator
#       province_rent_price_parts = [price.replace(',', '.') for price in province_rent_price_parts]
      
#       # print(index, province_sale_price_parts[0], province_sale_price_parts[1], province_rent_price_parts[0], province_rent_price_parts[1])
#       province_data.append([index, province_sale_price_parts[0], province_sale_price_parts[1], 
#                           province_rent_price_parts[0], province_rent_price_parts[1]])

#     else:
#       province_data.append([index, 0,0,0,0])

#     #Now load the municipality data
#     municipality_soup = province_soup.find_all("tr", class_="nd-table__row")
#     loop_counter = 0
#     for municipality_html in municipality_soup:
#         loop_counter += 1
#         municipality_name = ''
#         url = ''
#         sale = 0
#         rent = 0

#         for child in municipality_html.children:
#             if type(child) == Tag:
#                 if child.get("data-text") == "(€/m²)":
#                     link = child.find('a')
#                     if (link):
#                         url = link.get('href')
#                         municipality_name = link.text
#                         print(f'municipality_name: {municipality_name}')
#                 elif child.get("data-text") == "Sale":
#                     sale = child.text.replace('.', '')
#                 elif child.get("data-text") == "Rent":
#                     rent = child.text.replace(',', '.')
#         municipality_data.append([province['region'], province['province'], municipality_name, sale, rent, url])            
#     print(f'-----{loop_counter}::{len(municipality_data)}')
# print(f'municipality_data: {municipality_data.shape}')
# municipality_data_df = pd.DataFrame(data=municipality_data, columns=['region', 'province', 'city', 'sale_mean', 'rent_mean', 'url'])
# print(f'municipality_data_df: {municipality_data_df.shape}')

# province_data = pd.DataFrame(data=province_data, columns=['index', 'sale_low', 'sale_high', 'rent_low', 'rent_high'])
# province_data.set_index('index', inplace=True)
# # display(province_data)

# #merge province dataframes, reorder columns
# provinces_data_df = provinces_data_df.merge(province_data, left_index=True, right_index=True)
# display(provinces_data_df)
# provinces_data_df = provinces_data_df[['region', 'province', 'sale_low', 'sale_mean', 'sale_high', 'rent_low', 'rent_mean', 'rent_high', 'url']]
# print(f'provinces_data_df: {provinces_data_df.shape}')

# # provinces_data_df

# TO FIX:  "Valle d'Aosta" Has no provinces

This is causing a bug where the municipalites of the region are being listed multiple times

In [107]:
province_data = []
municipality_data = []

# print(f'shape: {regions_df.shape}')

#DBD todo: for the time being, we only care about Sicily, but this should be rewritten
for index, province in provinces_data_df[provinces_data_df['region'] == "Sicilia"].iterrows():
# for index, province in provinces_data_df[provinces_data_df['region'] == "Abruzzo"].iterrows():
# for index, province in provinces_data_df[provinces_data_df['region'] != "Valle d'Aosta"].iterrows():
# for index, province in provinces_data_df.iterrows():
    print(f'{index}: {province["url"]}')
    
    province_req = requests.get(province["url"], headers)
    province_soup = BeautifulSoup(province_req.content, 'html.parser')    
    
    #load the province price data
    
    province_prices = province_soup.find_all("p", class_="nd-cgHighlighted__subtext")
    
    #SALE 
    #break the string into words
    province_sale_price_parts = province_prices[0].contents[0].split()
    
    #keep only the parts we want
    province_sale_price_parts
    # print(f'province_sale_price_parts: {province_sale_price_parts}')
    if len(province_sale_price_parts) == 6:
      province_sale_price_parts = [province_sale_price_parts[i] for i in [1,4]]

      #remove '.' thousands separator
      province_sale_price_parts = [price.replace('.', '') for price in province_sale_price_parts]
      # print(f'Sale: {province_sale_price_parts}')
      
      #RENT 
      #break the string into words
      province_rent_price_parts = province_prices[1].contents[0].split()
      
      #keep only the parts we want
      province_rent_price_parts = [province_rent_price_parts[i] for i in [1,4]]
      # print(f'Rent: {province_rent_price_parts}')

      #remove '.' thousands separator
      province_rent_price_parts = [price.replace(',', '.') for price in province_rent_price_parts]
      
      # print(index, province_sale_price_parts[0], province_sale_price_parts[1], province_rent_price_parts[0], province_rent_price_parts[1])
      province_data.append([index, province_sale_price_parts[0], province_sale_price_parts[1], 
                          province_rent_price_parts[0], province_rent_price_parts[1]])

    else:
      province_data.append([index, 0,0,0,0])

    #Now load the municipality data
    municipality_soup = province_soup.find_all("tr", class_="nd-table__row")
    loop_counter = 0
    for municipality_html in municipality_soup:
        loop_counter += 1
        municipality_name = ''
        url = ''
        sale = 0
        rent = 0

        for child in municipality_html.children:
            if type(child) == Tag:
                if child.get("data-text") == "(€/m²)":
                    link = child.find('a')
                    if (link):
                        url = link.get('href')
                        municipality_name = link.text
                        print(f'municipality_name: {municipality_name}')
                elif child.get("data-text") == "Sale":
                    sale = child.text.replace('.', '')
                elif child.get("data-text") == "Rent":
                    rent = child.text.replace(',', '.')
        municipality_data.append([province['region'], province['province'], municipality_name, sale, rent, url])            
    # print(f'-----{loop_counter}::{len(municipality_data)}')

municipality_data_df = pd.DataFrame(data=municipality_data, columns=['region', 'province', 'city', 'sale_mean', 'rent_mean', 'url'])
# print(f'municipality_data_df: {municipality_data_df.shape}')

province_data = pd.DataFrame(data=province_data, columns=['index', 'sale_low', 'sale_high', 'rent_low', 'rent_high'])
province_data.set_index('index', inplace=True)
# display(province_data)

#merge province dataframes, reorder columns
provinces_data_df = provinces_data_df.merge(province_data, left_index=True, right_index=True)
display(provinces_data_df)
provinces_data_df = provinces_data_df[['region', 'province', 'sale_low', 'sale_mean', 'sale_high', 'rent_low', 'rent_mean', 'rent_high', 'url']]
# print(f'provinces_data_df: {provinces_data_df.shape}')

# provinces_data_df

77: https://www.immobiliare.it/en/mercato-immobiliare/sicilia/agrigento-provincia/
municipality_name: Agrigento
municipality_name: Alessandria della Rocca
municipality_name: Aragona
municipality_name: Bivona
municipality_name: Burgio
municipality_name: Calamonaci
municipality_name: Caltabellotta
municipality_name: Camastra
municipality_name: Cammarata
municipality_name: Campobello di Licata
municipality_name: Canicattì
municipality_name: Casteltermini
municipality_name: Castrofilippo
municipality_name: Cattolica Eraclea
municipality_name: Cianciana
municipality_name: Comitini
municipality_name: Favara
municipality_name: Grotte
municipality_name: Joppolo Giancaxio
municipality_name: Lampedusa e Linosa
municipality_name: Licata
municipality_name: Lucca Sicula
municipality_name: Menfi
municipality_name: Montallegro
municipality_name: Montevago
municipality_name: Naro
municipality_name: Palma di Montechiaro
municipality_name: Porto Empedocle
municipality_name: Racalmuto
municipality_name: 

,region,province,sale_mean,rent_mean,url,sale_low,sale_high,rent_low,rent_high
77,Sicilia,Agrigento,870,5.70,https://www.immobiliare.it/en/mercato-immobili...,288,2407,2.58,35.53
78,Sicilia,Caltanissetta,681,3.98,https://www.immobiliare.it/en/mercato-immobili...,339,836,3.82,5.97
79,Sicilia,Catania,1131,7.25,https://www.immobiliare.it/en/mercato-immobili...,305,1797,3.46,11.32
80,Sicilia,Enna,752,4.24,https://www.immobiliare.it/en/mercato-immobili...,298,888,3.72,4.54
81,Sicilia,Messina,1229,7.57,https://www.immobiliare.it/en/mercato-immobili...,233,3845,2.67,17.05
82,Sicilia,Palermo,1235,7.44,https://www.immobiliare.it/en/mercato-immobili...,201,2410,2.92,15.75
83,Sicilia,Ragusa,994,9.62,https://www.immobiliare.it/en/mercato-immobili...,329,1297,3.95,16.62
84,Sicilia,Siracusa,1115,9.45,https://www.immobiliare.it/en/mercato-immobili...,282,1624,3.43,12.57
85,Sicilia,Trapani,1045,5.76,https://www.immobiliare.it/en/mercato-immobili...,251,3204,4.89,10.02


In [108]:
provinces_data_df.to_csv('/drive/My Drive/Colab_Data/AirBnb/Italy/Sicily/derived/re_provinces.csv')

In [109]:
municipality_data_df.head()

,region,province,city,sale_mean,rent_mean,url
0,Sicilia,Agrigento,Agrigento,945,5.58,https://www.immobiliare.it/en/mercato-immobili...
1,Sicilia,Agrigento,Alessandria della Rocca,376,4.02,https://www.immobiliare.it/en/mercato-immobili...
2,Sicilia,Agrigento,Aragona,634,2.58,https://www.immobiliare.it/en/mercato-immobili...
3,Sicilia,Agrigento,Bivona,300,4.40,https://www.immobiliare.it/en/mercato-immobili...
4,Sicilia,Agrigento,Burgio,288,4.62,https://www.immobiliare.it/en/mercato-immobili...


In [110]:
municipality_data_df.shape

(390, 6)

# Load Municipality Detail

In [111]:
municipality_data = []
for index, municipality in municipality_data_df.iterrows():
    print(f'{index}: {municipality["url"]}')
    
    municipality_req = requests.get(municipality["url"], headers)
    municipality_soup = BeautifulSoup(municipality_req.content, 'html.parser')    
    
    #load the province price data
    
    municipality_prices = municipality_soup.find_all("p", class_="nd-cgHighlighted__subtext")
    
    #SALE 
    #break the string into words
    print(f'municipality_prices: {municipality_prices}')
    print(f'municipality_prices: {municipality_prices[0].contents}')
    municipality_sale_price_parts = municipality_prices[0].contents[0].split()
    
    #keep only the parts we want
    # print(f'municipality_sale_price_parts: {municipality_sale_price_parts}')
    if len(municipality_sale_price_parts) == 6:
      municipality_sale_price_parts = [municipality_sale_price_parts[i] for i in [1,4]]
      # print(f'municipality_sale_price_parts: {municipality_sale_price_parts}')

      #remove '.' thousands separator
      municipality_sale_price_parts = [price.replace('.', '') for price in municipality_sale_price_parts]
      # print(f'Sale: {municipality_sale_price_parts}')
      
      #RENT 
      #break the string into words
      municipality_rent_price_parts = municipality_prices[1].contents[0].split()
      
      #keep only the parts we want
      municipality_rent_price_parts = [municipality_rent_price_parts[i] for i in [1,4]]
      # print(f'Rent: {municipality_rent_price_parts}')

      #remove '.' thousands separator
      municipality_rent_price_parts = [price.replace(',', '.') for price in municipality_rent_price_parts]
      
      # print(index, province_sale_price_parts[0], province_sale_price_parts[1], province_rent_price_parts[0], province_rent_price_parts[1])
      municipality_data.append([index, municipality_sale_price_parts[0], municipality_sale_price_parts[1], 
                          municipality_rent_price_parts[0], municipality_rent_price_parts[1]])

    else:
      municipality_data.append([index, 0,0,0,0])

# print(f'municipality_data: {municipality_data.shape}')

0: https://www.immobiliare.it/en/mercato-immobiliare/sicilia/agrigento/
municipality_prices: [<p class="nd-cgHighlighted__subtext">from 806 €/m² to 1.262 €/m²</p>, <p class="nd-cgHighlighted__subtext">from 4,29 €/m² to 7,11 €/m²</p>]
municipality_prices: ['from 806 €/m² to 1.262 €/m²']
1: https://www.immobiliare.it/en/mercato-immobiliare/sicilia/alessandria-della-rocca/
municipality_prices: [<p class="nd-cgHighlighted__subtext">from 288 €/m² to 2.407 €/m²</p>, <p class="nd-cgHighlighted__subtext">from 2,58 €/m² to 35,53 €/m²</p>]
municipality_prices: ['from 288 €/m² to 2.407 €/m²']
2: https://www.immobiliare.it/en/mercato-immobiliare/sicilia/aragona/
municipality_prices: [<p class="nd-cgHighlighted__subtext">from 288 €/m² to 2.407 €/m²</p>, <p class="nd-cgHighlighted__subtext">from 2,58 €/m² to 35,53 €/m²</p>]
municipality_prices: ['from 288 €/m² to 2.407 €/m²']
3: https://www.immobiliare.it/en/mercato-immobiliare/sicilia/bivona/
municipality_prices: [<p class="nd-cgHighlighted__subtex

AttributeError: ignored

In [112]:
len(municipality_data)

390

In [113]:
md_df = pd.DataFrame(data=municipality_data, columns=['index', 'sale_low', 'sale_high', 'rent_low', 'rent_high'])
md_df.set_index('index', inplace=True)
md_df

,sale_low,sale_high,rent_low,rent_high
index,,,,
0,806,1262,4.29,7.11
1,288,2407,2.58,35.53
2,288,2407,2.58,35.53
3,288,2407,2.58,35.53
4,288,2407,2.58,35.53
...,...,...,...,...
385,251,3204,4.89,10.02
386,251,3204,4.89,10.02
387,251,3204,4.89,10.02


In [114]:
municipality_data = pd.DataFrame(data=municipality_data, columns=['index', 'sale_low', 'sale_high', 'rent_low', 'rent_high'])
municipality_data.set_index('index', inplace=True)
print(f'A: {municipality_data.shape}')
# display(municipality_data2)

#merge province dataframes, reorder columns
municipality_data_df = municipality_data_df.merge(municipality_data, left_index=True, right_index=True)
print(f'B: {municipality_data_df.shape}')
# display(municipality_data_df)
municipality_data_df = municipality_data_df[['region', 'province', 'city', 'sale_low', 'sale_mean', 'sale_high', 'rent_low', 'rent_mean', 'rent_high', 'url']]
municipality_data_df.to_csv('/drive/My Drive/Colab_Data/AirBnb/Italy/Sicily/derived/re_municipalities.csv')

municipality_data_df.head()

A: (390, 4)
B: (390, 10)


,region,province,city,sale_low,sale_mean,sale_high,rent_low,rent_mean,rent_high,url
0,Sicilia,Agrigento,Agrigento,806,945,1262,4.29,5.58,7.11,https://www.immobiliare.it/en/mercato-immobili...
1,Sicilia,Agrigento,Alessandria della Rocca,288,376,2407,2.58,4.02,35.53,https://www.immobiliare.it/en/mercato-immobili...
2,Sicilia,Agrigento,Aragona,288,634,2407,2.58,2.58,35.53,https://www.immobiliare.it/en/mercato-immobili...
3,Sicilia,Agrigento,Bivona,288,300,2407,2.58,4.40,35.53,https://www.immobiliare.it/en/mercato-immobili...
4,Sicilia,Agrigento,Burgio,288,288,2407,2.58,4.62,35.53,https://www.immobiliare.it/en/mercato-immobili...
